# Application B

In [2]:
import yaml
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from pychromatic import Multiplot
from pychromatic.colors import accent

Run bcc calculations

In [22]:
with open("input.yaml","r") as fin:
    inp = yaml.load(fin, Loader=yaml.FullLoader)

Modify the potential pathmax_line_width

In [23]:
potential = os.path.join(os.getcwd(), "Ti.eam.fs")
inp["md"]["pair_coeff"] = "* * %s Ti"%potential

Now we have to add bcc specific options

In [24]:
press = np.arange(0, 50000+2500, 2500)

In [25]:
inp["calculations"][0]["lattice"][0] = "BCC"
inp["calculations"][0]["lattice_constant"] = float(3.251)
inp["calculations"][0]["temperature"] = [int(500), int(2250)]
inp["queue"]["jobname"] = "bcc"

In [26]:
calcs = []
for p in press:
    calctemplate = inp["calculations"][0].copy()
    calctemplate["pressure"] = int(p)
    calcs.append(calctemplate)

In [27]:
inp["calculations"] = calcs

Save the input file

In [28]:
with open("input-bcc.yaml", "w", encoding="utf-8") as fout:
    yaml.dump(inp, fout)

Now for hcp structure

In [10]:
with open("input.yaml","r") as fin:
    inp = yaml.load(fin, Loader=yaml.FullLoader)

In [11]:
potential = os.path.join(os.getcwd(), "Ti.eam.fs")
inp["md"]["pair_coeff"] = "* * %s Ti"%potential

Now we have to add hcp specific options

In [12]:
inp["calculations"][0]["lattice"][0] = "HCP"
inp["calculations"][0]["temperature"] = [int(500), int(1500)]
inp["calculations"][0]["iso"] = "False"
inp["queue"]["jobname"] = "hcp"

In [13]:
calcs = []
for p in press:
    calctemplate = inp["calculations"][0].copy()
    calctemplate["pressure"] = int(p)
    calcs.append(calctemplate)

In [14]:
inp["calculations"] = calcs

Save the input file

In [15]:
with open("input-hcp.yaml", "w", encoding="utf-8") as fout:
    yaml.dump(inp, fout)

Now liquid

In [16]:
with open("input.yaml","r") as fin:
    inp = yaml.load(fin, Loader=yaml.FullLoader)

In [17]:
potential = os.path.join(os.getcwd(), "Ti.eam.fs")
inp["md"]["pair_coeff"] = "* * %s Ti"%potential

Now we have to add hcp specific options

In [18]:
inp["calculations"][0]["lattice"][0] = "LQD"
inp["calculations"][0]["temperature"] = [int(1500), int(3000)]
inp["calculations"][0]["state"] = "liquid"
inp["queue"]["jobname"] = "lqd"

In [19]:
calcs = []
for p in press:
    calctemplate = inp["calculations"][0].copy()
    calctemplate["pressure"] = int(p)
    calcs.append(calctemplate)

In [20]:
inp["calculations"] = calcs

Save the input file

In [21]:
with open("input-lqd.yaml", "w", encoding="utf-8") as fout:
    yaml.dump(inp, fout)

Calculations are done, now we can extract the coexistence temperature. 